Effet Tunnel

In [1]:
#import
import math as math
import numpy as np
import matplotlib as plt
print("import done")

import done


Fourrier transform

In [2]:
def g(x):
    g=np.exp(-x**2)
    return g

def mu_ech(f):
    k=1000 #nombre d'instances
    T=0.1
    dT=0.2 # doit être inférieur à T
    for k in range:
        dTe

def F(f,mu):
    k0=1000000
    mu_ech=1
    sum=0
    for k in range (-k0,k0+1):
        sum=sum+f(k)*np.exp(-2*1j*np.pi*k*mu/mu_ech)
    return sum

print(F(g,4))
print(g(4))

(1.7726372048266523-2.603731301266414e-32j)
1.1253517471925912e-07


Normalement la fonction final

In [ ]:
def Schrodinger(r,t):
    S0 = Schrod0(r)
    Et = U(t)
    return Et*S0

l'opérateur d'énergie potentielle V

In [ ]:
def V(r):
    return 
